### Read cryo DAT files and plots X,Y,Z components

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal
from tkinter import *
from tkinter import filedialog

In [2]:
'''
read SAMPLE INFORMATION AND DEMAG DATA from binary DAT file from magnetometer (format of 2G-Barcelona)
'''
def readDATfile(filename):                    
    with open(filename, 'r') as f:
        data = f.read()
        sample={}
        sample["name"]= data[15:24].rstrip('\x00')                #sample name
        #sample["vol"]= "{:5.2f}".format(float(data[24:29].strip().strip('\x00')))   #sample volume
        sample["vol"]= float(data[24:29].strip().strip('\x00'))   #sample volume
        sample["comment"] = data[30:57].strip().strip('\x00')
        sample["CoreAz"] = float(data[111:114].strip().strip('\x00'))       # in- CoreAzimut
        sample["CoreInc"] = float(data[114:118].strip().strip('\x00'))      #az - CoreInc
        sample["BedAz"] = float(data[118:123].strip('\x00'))                #str - Bed dip directiom
        sample["BedInc"] = float(data[123:127].strip('\x00'))               #dip - Bed inclination
        sample["mag_Dec"] = float(data[142:147].strip('\x00'))
        sample["com"] = 50                                      #50 microTesla

        sample_demag=[]
        measurement = {}
        for n in range(len(data)):
            if data[n]=="Í":
                line = n+1
                step = data[line:line+4].strip().strip('\x00')
                treatment = data[line+4:line+5].strip().strip('\x00')
                X = float(data[line+65:line+76].strip().strip('\x00'))/1E+3     # Am2  
                Y = float(data[line+97:line+108].strip().strip('\x00'))/1E+3    # Am2
                Z = float(data[line+129:line+140].strip().strip('\x00'))/1E+3   # Am2
                Intensity = np.sqrt(X**2 + Y**2 + Z**2)
                measurement ={"step":step,"treat":treatment,
                          "X":X,"Y":Y,"Z":Z, "Int (Am2)":Intensity}
                sample_demag.append(measurement)
    return(sample,sample_demag)

In [3]:
'''
WRITES (ADD) SAMPLE INFORMATION TO TXT FILE
'''
def WriteTXT_heading_Tellier(sample,output):
    with open(output, 'a') as f:        # a= to append (not overwritting)
        heading_line = "Id:{0}in:{1:05.1f} az:{2:04.1f}  dip:{3:04.1f}  str:{4:05.1f} m:{5:05.2f} com:{6}".format(
                                        sample["name"].ljust(13),sample["CoreAz"],sample["CoreInc"],
                                        sample["BedInc"],sample["BedAz"],sample["vol"],sample["com"])
        print(heading_line)
        print()
        f.write(heading_line)
        f.write('\n')
        f.write('\n')    

In [4]:
'''
WRITES DEMAG DATA TO TXT FILE
'''    
def WriteTXT_steps_Tellier(sample_demag, step,output):
    global step_index
    step_index = step
    with open(output, 'a') as f:
        for n in range(len(sample_demag)):
            if sample_demag[n]['step'] == 'NRM':
                step = "0N0"
            if sample_demag[n]['step'][-1:] == '0':
                step_index += 1
                step = sample_demag[n]['step']+'R'+chr(64+step_index)  #chr(65) is 'A'
            if sample_demag[n]['step'][-1:] == '1':
                step = str(int(sample_demag[n]['step'])-1)+'V'+chr(64+step_index)
            if sample_demag[n]['step'][-1:] == '2':
                step = str(int(sample_demag[n]['step'])-2)+'P'+chr(64+step_index)
            if sample_demag[n]['step'][-1:] == '3':
                step = str(int(sample_demag[n]['step'])-3)+'X+'
            if sample_demag[n]['step'][-1:] == '4':
                step = str(int(sample_demag[n]['step'])-4)+'X-'
            if sample_demag[n]['step'][-1:] == '5':
                step = str(int(sample_demag[n]['step'])-5)+'Y+'
            if sample_demag[n]['step'][-1:] == '6':
                step = str(int(sample_demag[n]['step'])-6)+'Y-'
            if sample_demag[n]['step'][-1:] == '7':
                step = str(int(sample_demag[n]['step'])-7)+'Z+'

            step_line = "{0}{1:11}{2:11}{3:11}   1 Mo     0.".format(step.rjust(7),
                                                                     str("%.3E" % Decimal(sample_demag[n]['X'])).rjust(11),
                                                                     str("%.3E" % Decimal(sample_demag[n]['Y'])).rjust(11),
                                                                     str("%.3E" % Decimal(sample_demag[n]['Z'])).rjust(11))
            print(step_line)
            f.write(step_line)
            f.write('\n')

### Selecting a collection of files

In [5]:
root = Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)
root.title("select first set of files")
filenames_1part = filedialog.askopenfilenames(filetypes = (('binary DAT','typeName {DAT}')))
filenames_1part
filenames_2part = filedialog.askopenfilenames(filetypes = (('binary DAT','typeName {DAT}')))
filenames_1part, filenames_2part

(('C:/Users/mgarc/OneDrive - Universitat de Barcelona/GitHub/paleomagnetism/arqueomag/dat_arqueomag/fins420/1b-3b.dat',
  'C:/Users/mgarc/OneDrive - Universitat de Barcelona/GitHub/paleomagnetism/arqueomag/dat_arqueomag/fins420/2a-2a.dat',
  'C:/Users/mgarc/OneDrive - Universitat de Barcelona/GitHub/paleomagnetism/arqueomag/dat_arqueomag/fins420/3a-2a.dat',
  'C:/Users/mgarc/OneDrive - Universitat de Barcelona/GitHub/paleomagnetism/arqueomag/dat_arqueomag/fins420/4b-3b.dat'),
 ('C:/Users/mgarc/OneDrive - Universitat de Barcelona/GitHub/paleomagnetism/arqueomag/dat_arqueomag/1b-3b.dat',
  'C:/Users/mgarc/OneDrive - Universitat de Barcelona/GitHub/paleomagnetism/arqueomag/dat_arqueomag/2a-2a.dat',
  'C:/Users/mgarc/OneDrive - Universitat de Barcelona/GitHub/paleomagnetism/arqueomag/dat_arqueomag/3a-2a.dat',
  'C:/Users/mgarc/OneDrive - Universitat de Barcelona/GitHub/paleomagnetism/arqueomag/dat_arqueomag/4b-3b.dat'))

In [6]:
output = str(input("Site name: "))
outputfile = output+"Tellier.TXT"
outputfile

Site name: prueba_


'prueba_Tellier.TXT'

In [7]:
global step_index
step = 0

for file in range(len(filenames_1part)):
    sample,sample_demag=readDATfile(filenames_1part[file])
    WriteTXT_heading_Tellier(sample, outputfile)
    WriteTXT_steps_Tellier(sample_demag, step, outputfile)
    if filenames_2part:
        step = step_index
        sample,sample_demag=readDATfile(filenames_2part[file])
        WriteTXT_steps_Tellier(sample_demag, step, outputfile)


Id:1b-3b        in:004.0 az:90.0  dip:00.0  str:000.0 m:08.00 com:50

    0N0  1.800E-06 -1.043E-06  2.521E-06   1 Mo     0.
  100RA  1.638E-06 -1.060E-06  2.683E-06   1 Mo     0.
  100VA  1.609E-06 -1.108E-06  2.490E-06   1 Mo     0.
  150RB  1.529E-06 -9.378E-07  2.742E-06   1 Mo     0.
  150VB  1.558E-06 -9.909E-07  2.348E-06   1 Mo     0.
  200RC  1.506E-06 -9.290E-07  2.702E-06   1 Mo     0.
  200VC  1.496E-06 -9.233E-07  2.090E-06   1 Mo     0.
  100PC  1.460E-06 -1.004E-06  2.241E-06   1 Mo     0.
  250RD  1.369E-06 -9.172E-07  2.682E-06   1 Mo     0.
  250VD  1.253E-06 -8.165E-07  1.851E-06   1 Mo     0.
  300RE  1.173E-06 -8.620E-07  2.726E-06   1 Mo     0.
  300VE  1.247E-06 -7.416E-07  1.265E-06   1 Mo     0.
  200PE  1.200E-06 -8.415E-07  1.909E-06   1 Mo     0.
  330RF  1.070E-06 -7.575E-07  2.741E-06   1 Mo     0.
  330VF  1.084E-06 -6.635E-07  1.011E-06   1 Mo     0.
  360RG  7.793E-07 -5.843E-07  2.848E-06   1 Mo     0.
  360VG  1.026E-06 -5.976E-07  5.574E-07   1 Mo   

In [8]:
#UNMARK TO PRINT THE LIST OF BYTES
'''
# Create Tk root
root = Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)
filename = filedialog.askopenfilename(filetypes = (('binary DAT','typeName {DAT}')))
 

try:
    with open(filename, "rb") as f:
        byte = f.read(1)
        sum_bytes = np.array(byte)
        n=1
        while byte:
            # Do stuff with byte.
            byte = f.read(1)
            sum_bytes = np.append(sum_bytes, byte)
            print(n, byte)
            n += 1
except IOError:
     print('Error While Opening the file!')
'''

'\n# Create Tk root\nroot = Tk()\n# Hide the main window\nroot.withdraw()\nroot.call(\'wm\', \'attributes\', \'.\', \'-topmost\', True)\nfilename = filedialog.askopenfilename(filetypes = ((\'binary DAT\',\'typeName {DAT}\')))\n \n\ntry:\n    with open(filename, "rb") as f:\n        byte = f.read(1)\n        sum_bytes = np.array(byte)\n        n=1\n        while byte:\n            # Do stuff with byte.\n            byte = f.read(1)\n            sum_bytes = np.append(sum_bytes, byte)\n            print(n, byte)\n            n += 1\nexcept IOError:\n     print(\'Error While Opening the file!\')\n'